In [1]:
import pandas as pd
import re,itertools
from nltk.corpus import stopwords
flatten = itertools.chain.from_iterable
import spacy  
import numpy as np
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases

from gensim.models import Phrases
from gensim.models.phrases import Phraser

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [57]:
from phrase_extract import custom_ner
from random import shuffle

In [4]:
stopword_list = stopwords.words('english')
stopword_list.extend(['google','facebook','twitter','linkedin','whatsapp'])

In [5]:
df = pd.read_csv('raw_blog_content_cleaned.csv')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,title,url,body
0,Understanding Business Development,http://sethgodin.typepad.com/seths_blog/2009/0...,Understanding business development. Business D...
1,What are different types of Marketing Research?,http://whatismarketresearch.com/market-researc...,What are different types of Marketing Research...
2,5 Low-Cost Marketing Research Techniques for S...,https://www.credibly.com/incredibly/5-low-cost...,5 Low-Cost Marketing Research Techniques for S...
3,Are Product Managers the New Software Engineers?,https://hired.com/blog/highlights/product-mana...,Are Product Managers the New Software Engineer...
4,COMPETITIVE ANALYSIS PART 2: HOW TO ANALYZE C...,https://www.campaigncreators.com/blog/competit...,Competitive Analysis Part 2: How to Analyze Co...


In [6]:
def preProcess_stage1(text):
    #print "original:", text
    # sentence segmentation - assume already done
    # Remove Emails
    text = re.sub(r'\S*@\S*\s?', '', text)
    # Remove website links
    text = re.sub(r'http[s]?://\S+', '', text)
    # Remove distracting single quotes
    text = re.sub(r"\'", "", text)
    # Remove distracting double quotes
    text = re.sub(r'\"', "", text)
    # Remove new line characters
    text = re.sub(r'\s+', ' ', text)
    # word normalisation
    text = re.sub(r"(\w)([.,;:!?'/\"”\)])", r"\1 \2", text)
    text = re.sub(r"([.,;:!?'/\"“\(])(\w)", r"\1 \2", text)
    # normalisation
    text = re.sub(r"(\S)\1\1+",r"\1\1\1", text)
    #tokenising
    
    tokens = list(flatten([re.split(r"\s+",t) for t in re.split('(\d+)',text)]))
    tokens = [re.sub(r'[^A-Za-z]+','',t) for t in tokens]
    tokens = [t.lower() for t in tokens]
    tokens = [t for t in tokens if t not in ' ']
    return tokens

In [7]:
# corpus of all body data for NER
with open('corpus.txt','w') as f:
    allblog = df.body.values.tolist()
    for blog in allblog:
        sentList = []
        sentences = blog.split('.')
        for sent in sentences:
            tokens = preProcess_stage1(sent)
            sentList.append(' '.join(tokens))
        f.writelines('. '.join(sentList))

# stopword file for NER
stopword_file=open('stopword.txt','w')
stopword_file.writelines(stopword_list)
stopword_file.close()



In [8]:
df['tokens_1'] = df.apply(lambda x : preProcess_stage1(str(x.body)),axis = 1)

In [9]:
df['new_body'] = df.apply(lambda x : '. '.join([' '.join(preProcess_stage1(sent)) for sent in x.body.split('.')]),axis = 1)

In [10]:
df['ner_terms'] = df.apply(lambda x : custom_ner(x.new_body),axis = 1)

In [11]:
df['ner_terms']

0      [mysterious title, discussed function, departm...
1      [marketing research marketers, marketing resea...
2      [dollars, focus groups, connect, customers, un...
3      [product managers, vaunted kings, silicon vall...
4      [competitive analysis, analyze competitive inf...
5      [technical skills, growth hackers, learn techn...
6      [importance, lead generation criteria, success...
7      [process, gathering analyzing, interpreting in...
8      [agile kanban, agileto, agile scrum, methodolo...
9      [james doyle, definition, ambitious, years ago...
10     [mergers, acquisitions, complete guide cleveri...
11     [digital media, changing, important, startups,...
12     [recently helped, prioritize, measurement fram...
13     [hybrid ideal, harvard business school, echoin...
14     [beginners guide, combine, buyer profile, buyi...
15     [adeo ressi, cofounder, founder institute, maj...
16     [engineering, traction explains, traction chan...
17     [core tenets, core tenet

In [12]:
nlp = spacy.load('en', disable=['parser', 'ner'])

def remove_stopwords(tokens):
    return [t for t in tokens if t not in stopword_list]

def make_bigrams(tokens):
    tokens = [tokens]
    bigram = Phrases(tokens, min_count=1, threshold=2, delimiter=b' ')
    bigram_phraser = Phraser(bigram)
    bigram_tokens = list(flatten([bigram_phraser[sent] for sent in tokens]))
    bigrams_new = [t for t in bigram_tokens if len(t.split()) > 1 ]
#     bigrams_new = ['_'.join(t.split())  if len(t.split()) > 1 else t for t in bigram_tokens]
    return bigrams_new

def make_trigrams(tokens):
    tokens = [tokens]
    bigram = gensim.models.Phrases(tokens, min_count=1, threshold=2) 
    trigram = gensim.models.Phrases(bigram[tokens], threshold=2) 
    trigram_phraser = Phraser(trigram)
    trigram_tokens = list(flatten([trigram_phraser[sent] for sent in tokens]))
    trigrams_new = [t for t in trigram_tokens if len(t.split()) > 2 ]
    return trigrams_new

def lemmatization(tokens, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    doc = nlp(" ".join(tokens)) 
    return [token.lemma_ for token in doc if token.pos_ in allowed_postags]

def preProcess_stage2(tokens):
    # Stop Words
    clean_tokens = remove_stopwords(tokens)

    # BIgrams
    bigrams_tokens = make_bigrams(clean_tokens)

    # TRIgarms
    trigrams_tokens = make_trigrams(clean_tokens)
    
    # remove lemma with noun, adj, vb, adv words only
#     token_lemma = lemmatization(bigrams_tokens, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    token_lemma = lemmatization(clean_tokens, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    token_lemma = [t for t in token_lemma if len(t) > 2 ]
    return token_lemma + bigrams_tokens + trigrams_tokens

In [13]:
df['tokens_2'] = df.apply(lambda x : preProcess_stage2(x.tokens_1),axis = 1)

In [14]:
print(df['tokens_2'].head())

0    [understand, business, development, business, ...
1    [different, type, market, research, marketer, ...
2    [lowcost, marketing, research, technique, smal...
3    [product, manager, new, software, engineer, so...
4    [competitive, analysis, part, analyze, competi...
Name: tokens_2, dtype: object


In [15]:
data_lemmatized = df.tokens_2.values.tolist()
data_lemmatized = df.ner_terms.values.tolist()
# data_lemmatized[:1]
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
# print(corpus[:1])

In [16]:
# id2word[0]

In [17]:
# # Human readable format of corpus (term-frequency)
# [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [18]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [19]:
# Build LDA Multicore model
lda_multicore = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                           num_topics=20,
                           id2word=id2word,
                           workers=3)

In [20]:
# Build LDA Mallet model
mallet_path = 'mallet-2.0.8/bin/mallet' 
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [21]:
# # Print the Keyword in the 10 topics
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

In [22]:
# Compute Perplexity of LDA
print('\nPerplexity of LDA: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score of LDA
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score of LDA: ', coherence_lda)

# Compute Perplexity of LDA Multicore
print('\nPerplexity of LDA Multicore: ', lda_multicore.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score of LDA Multicore
coherence_model_lda = CoherenceModel(model=lda_multicore, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score of LDA Multicore: ', coherence_lda)


# # Compute Perplexity of LDA Mallet
# print('\nPerplexity of LDA Mallet: ', ldamallet.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score of LDA Mallet
coherence_model_lda = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score of LDA Mallet: ', coherence_lda)



Perplexity of LDA:  -12.341826361567048

Coherence Score of LDA:  0.42752400902764737

Perplexity of LDA Multicore:  -12.121716275357466

Coherence Score of LDA Multicore:  0.3135491237497541

Coherence Score of LDA Mallet:  0.40844511946240825


In [23]:
# # Plotting tools
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
# import matplotlib.pyplot as plt
# %matplotlib inline

In [24]:
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# vis

In [25]:
optimal_model = ldamallet
data = df.body.values.tolist()
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'body']

# Show
df_dominant_topic.head(50)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,body
0,0,9.0,0.1676,"company, companies, platform, partners, experi...",Understanding business development. Business D...
1,1,19.0,0.3454,"research, information, marketing, questions, c...",What are different types of Marketing Research...
2,2,19.0,0.2490,"research, information, marketing, questions, c...",5 Low-Cost Marketing Research Techniques for S...
3,3,14.0,0.1620,"product, problem, products, features, designer...",Are Product Managers the New Software Engineer...
4,4,4.0,0.3782,"business, industry, competitors, specific, com...",Competitive Analysis Part 2: How to Analyze Co...
5,5,7.0,0.4500,"potential, finding, understand, attention, pop...",7 Technical Skills All Growth Hackers Should L...
6,6,10.0,0.3932,"business, marketing, businesses, account, clie...",The Importance of Lead Generation | Criteria f...
7,7,19.0,0.4977,"research, information, marketing, questions, c...",Market Research Definition - Entrepreneur Smal...
8,8,8.0,0.4854,"development, process, project, software, decis...",Kanban and Agile Kanban is a set of tools you...
9,9,9.0,0.1900,"company, companies, platform, partners, experi...",Product Development: Turning Months Into Weeks...


In [26]:
# print (pd.merge(df, df_dominant_topic, on='body'))
com = pd.merge(df, df_dominant_topic, on='body')

In [27]:
com.to_csv('lda_keyword.csv',index=False)

In [28]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [29]:
topics_scores = {
    "marketing" : 0.0,
    "branding" : 0.0,
    "growth marketing" : 0.0,
    "brand development" : 0.0,
    "growth strategies" : 0.0,
    "product management" : 0.0,
    "product discovery" : 0.0,
    "product growth" : 0.0,
    "product management fundamentals" : 0.0,
    "agile principles" : 0.0,
    "company culture" : 0.0,
    "company growth" : 0.0,
    "people management" : 0.0,
    "startup fundamentals" : 0.0,
    "interpersonal skills" : 0.0,
    "business fundamentals" : 0.0,
    "business growth" : 0.0,
    "sales growth" : 0.0,
    "investment cycle" : 0.0
}

In [61]:
#load word2vec model, here GoogleNews is used
#model = gensim.models.KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)
#two sample sentences 

def distance_metrics(x):
    #tokens = topics.split(',')
    topics = x.ner_terms
    for k,v in topics_scores.items():
        distance = []
        for t in topics:           
            #calculate distance between two sentences using WMD algorithm
            distance.append(model.wmdistance(k, t))
        topics_scores[k] = np.mean(distance)
    sorted_weight = sorted(topics_scores.items(), key=lambda x:x[1], reverse=True)
    return sorted_weight

def most_rated_topics(x):
    title_ner = custom_ner(x.title)
    final_tokens = []
    if title_ner:
        for t in title_ner:
            final_tokens.append(' '.join(preProcess_stage1(t)))
    #print(final_tokens)
    top_rated = list(x.assorted_topic_scores)[:5] 
    top_rated = final_tokens + [i[0] for i in top_rated]
    shuffle(top_rated)
    #print(top_rated)
    return ', '.join(top_rated)


In [33]:
com['assorted_topic_scores'] = com.apply(lambda x : distance_metrics(x), axis = 1 )

In [34]:
# com['topic']=com.apply(lambda x : most_relevant_topics(x.assorted_topic_scores), axis =1)
print(list(com['assorted_topic_scores'][1]))

[('branding', 2.0503522580800073), ('company growth', 1.8652958884797952), ('people management', 1.7791607668905323), ('product growth', 1.7612669196325632), ('marketing', 1.757534900256922), ('growth marketing', 1.7541899557768248), ('sales growth', 1.6759011699808894), ('business fundamentals', 1.6667830422844447), ('agile principles', 1.6567528658416975), ('business growth', 1.6530960062390432), ('brand development', 1.6439085205769481), ('startup fundamentals', 1.6406964718634272), ('company culture', 1.6292207432345744), ('product discovery', 1.6114807974715488), ('investment cycle', 1.5852816473451072), ('growth strategies', 1.572425141700845), ('interpersonal skills', 1.5695241333259642), ('product management fundamentals', 1.543130101098816), ('product management', 1.504305896961552)]


In [62]:
com['topic'] = com.apply(lambda x : most_rated_topics(x), axis = 1)

In [63]:
com['topic'][0]

'product growth, marketing, branding, growth marketing, understanding business development, company growth'

In [37]:
com.to_csv('lda_keyword.csv',index=False)

In [64]:
com[['title','url','topic']].to_json('articles_topic.json',orient='records')